<a href="https://colab.research.google.com/github/pavelpryadokhin/AI-Assistant/blob/main/AI_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Нейро-сотрудники

Разработаем различных нейропомощников. В качестве модели будем использовать chatGPT, для векторной базы данных выберем Chromadb, а фреймворк — langchain.

In [ ]:
!pip install openai gradio tiktoken langchain langchain-openai langchain-community chromadb sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.5/612.5 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 24.3 MB/s eta 0:0

In [ ]:
import getpass  # Для работы с паролями
import os  # Для работы с окружением и файловой системой

# Блок библиотек фреймворка LangChain
from langchain.docstore.document import Document  # Работа с документами
from langchain.embeddings.openai import OpenAIEmbeddings  # Эмбеддинги для OpenAI
from langchain.vectorstores import Chroma  # Доступ к векторной базе
from langchain.text_splitter import CharacterTextSplitter, MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter  # Разделение текста

from sentence_transformers.cross_encoder import CrossEncoder  # Cross Encoder

import requests  # Отправка запросов
from openai import OpenAI  # Доступ к OpenAI

import gradio as gr  # Интерфейс с помощью Gradio
import tiktoken  # Подсчет токенов
import re  # Регулярные выражения
import numpy as np  # Работа с массивами

# Запрос ввода ключа от OpenAI
os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key: ")

Введите OpenAI API Key:··········


In [ ]:
models = [
              {
                "doc": "https://docs.google.com/document/d/1smBnM5G_EjS0C6abDQXn5DlzCjk2ldrOTXf-ftoT7dM/edit",
                "prompt": '''Ты - Нейро-нутрициолог. Твоя задача - помогать пользователям отвечать на вопросы по
                 поводу питания и составлять индивидуальные меню. Нужно строго следовать следующим инструкциям:
                 1.Используй данную документацию как основное руководство. В ней содержится важная информация о
                 том, как считать и представлять данные, а также примеры меню, которым необходимо строго следовать.

                 2.При составление меню необходимо убедиться, что общее количество белков, жиров, углеводов и калорий (БЖУК)
                 в меню строго соответствует запросу с погрешностью не более 100ккал. Для расчета БЖУК следуй этому процессу:
                    # Подсчитай БЖУК каждого продукта, основываясь на его весе.
                    # Сложи БЖУК всех продуктов вместе, чтобы получить общее значение.

                 3.При презентации данных необходимо представлять информацию ясно и структурированно.
                 Обязательно включай данные о БЖУК и калорийности для каждого продукта и всего меню в целом.

                Строго следи за точностью расчетов и  представлением информации. Твоя помощь важна для достижения
                успешных результатов в питании пользователей.
                 Документация: ''',
                "name": "Нейро-нутрициолог (составление меню)",
                "query": "Составь меню на неделю на 2000 калорий в день"
              },
              {
                "doc": "https://docs.google.com/document/d/1f7Gfv2PZYACD1PGzlonfBZsWI9Pf9ZOFI_xYp3DRxw0/edit",
                "prompt": '''Ты менеджер поддержки hr отдела, к тебе могут обращаться hr менеджеры и рекрутеры за подсказками и ответами на их вопросы в чате компании.
                        Постарайся дать развернутый ответ, твоя задача ответить так, чтобы у менеджера не осталось больше вопросов к тебе.
                        Отвечай по существу, без лишних эмоций и слов, от тебя нужна только точная информация.
                        Отвечай максимально точно по документу, не придумывай ничего от себя.
                        Документ  с информацией для ответа клиенту: ''',
                "name": "Нейро-менеджер поддержки HR-отдела",
                "query": "Для чего нужны шаблоны вакансий?"
              },
              {
                "doc": "https://docs.google.com/document/d/1joE-rKrEmDgQojonmkyTYRihQ6eaoKraqgK_Pav5uLA/edit",
                "prompt": '''Ты сотрудник по подбору персонала. Перед тобой документ, в котором описана инструкция по подбору и оценке на должность руководителя отдела продаж.
                        Твоя задача придумывать вопросы к собеседованию на данную позицию.
                        Документ: ''',
                "name": "Нейро-рекрутер",
                "query": "Придумай 5 вопросов для собеседования"
              },
              {
                "doc": "https://docs.google.com/document/d/1SVi7dVwgJKf7tOljTWlVG-XebYXEv3B0Xjq8ERqjlbA/edit",
                "prompt": '''Перед тобой диалог преподавателя музыкальной школы с родителем ученика.
                        Тебе надо проверить несколько критериев и заполнить отчёт для занесения в календарь уроков.

                        Что надо проверить
                        1. Говорил ли родитель о потребности в обучении
                        2. На каком интрументе умеет играть ребенок
                        3. На какой инструмент записался
                        4. Когда первый пробный урок

                        Не пиши общее сообщение, только заполни отчёт по форме:
                        1. Говорил ли родитель о потребности в обучении - было или нет
                        2. На каком интрументе умеет играть ребенок - название инструмента
                        3. На какой инструмент записался ребенок - название инструмента
                        4. Когда первый пробный урок - на какое время договорились

                        Заполни отчёт и пришли в качестве ответа, коротко и ёмко ''',
                "name": "Нейро-менеджер (Отчет по диалогу)",
                "query": "Пришли отчет"
               },
              {
                "doc": "https://docs.google.com/document/d/1IqGa92RlFiCJvBH7TBKhPpODpru2-RDro8qiVEzoAuA/edit",
                "prompt": '''Ты менеджер контроля качества, твоя задача анализировать диалоги менеджеров по продажам с клиентами и готовить отчеты.
                        Компания продает курсы по машинному обучению.
                        Перед тобой текст диалога сделанный с помощью распознавания речи из записи zoom презентации.
                        Из-за машинного распознавания речи, в тексте могут быть ошибки распознавания, учитывая это.
                        Твоя задача делать отчеты по данному диалогу по запросам пользователя.
                        Составляй вопросы максимально точно по диалогу, не придумывай ничего от себя.
                        Текст диалога: ''',
                "name": "Нейро-менеджер контроля качества (Оценка качества по диалогу)",
                "query": "Напиши отчет, какие были потребности названы клиентом"
              }

]

##Оптимизация



В этом нейро-сотруднике были изменены функции, отвечающие за разбиение текста. Вместо CharacterTextSplitter, который ранее использовался, был внедрен MarkdownHeaderTextSplitter. Он обеспечивает более осмысленное деление текста на части. Если длина фрагмента оказывалась больше заданной, то к нему применялся метод RecursiveCharacterTextSplitter для дальнейшего разбиения.

Также в поддокументы были добавлены названия глав и подглав. Это позволило сделать поиск более точным и понятным.

Кроме того, были улучшены результаты поиска из векторной базы данных. Из восьми найденных документов с помощью Cross Encoder было отобрано только пять лучших.

## Код модели

In [ ]:
# Объявляем класс нейро-сотрудника
class GPT:
    def __init__(self, model="gpt-3.5-turbo"):
        self.log = ''  # Логи сообщений
        self.model = model  # Модель OpenAI
        self.search_index = None  # База знаний (None - модель не обучена)
        self.client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])  # Инициализация клиента OpenAI

    def load_search_indexes(self, url):
        # Извлечение document ID из URL
        match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
        if match_ is None:
            raise ValueError('Неверный Google Docs URL')

        doc_id = match_.group(1)  # ID документа
        response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=markdown')
        response.raise_for_status()  # Проверка статуса запроса

        text = response.text  # Извлечение текста
        return self.create_embedding(text)  # Создание эмбеддинга

    def num_tokens_from_string(self, string):
        """Возвращает число токенов в строке"""
        encoding = tiktoken.encoding_for_model(self.model)  # Кодировщик модели
        return len(encoding.encode(string))  # Число токенов

    def create_embedding(self, data):
        max_size = 1000  # Длина чанка
        source_chunks = []  # Список докуметнов
        headers_to_split_on = [('#', 'H1'), ('##', 'H2'), ('###', 'H3')]

        splitter_markdown = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=True)
        splitter_recurs = RecursiveCharacterTextSplitter(separators="\n", chunk_size=max_size, chunk_overlap=20)

        for chunk in splitter_markdown.split_text(data):
            headers = '\n'.join(chunk.metadata.values()) + '\n'
            if len(chunk.page_content) > max_size:
                for sub_chunk in splitter_recurs.split_text(chunk.page_content):
                    source_chunks.append(Document(page_content=headers + sub_chunk, metadata=chunk.metadata))
            else:
                source_chunks.append(Document(page_content=headers + chunk.page_content, metadata=chunk.metadata))

        count_token = self.num_tokens_from_string(' '.join([x.page_content for x in source_chunks]))
        self.log += f'Количество токенов в документе: {count_token}\n'

        self.search_index = Chroma.from_documents(source_chunks, OpenAIEmbeddings())
        self.log += 'Данные загружены в векторную базу данных.\n'

        return self.search_index

    def num_tokens_from_messages(self, messages, model):
        """Возвращает число токенов из списка сообщений"""
        try:
            encoding = tiktoken.encoding_for_model(model)  # Кодировщик модели
        except KeyError:
            print("Предупреждение: модель не создана. Используйте cl100k_base кодировку.")
            encoding = tiktoken.get_encoding("cl100k_base")  # Базовый кодировщик

        # Выбор модели и токенов
        if model in {
            "gpt-3.5-turbo-0613",
            "gpt-3.5-turbo-16k-0613",
            "gpt-4-0314",
            "gpt-4-32k-0314",
            "gpt-4-0613",
            "gpt-4-32k-0613",
            "gpt-4o",
            "gpt-4o-2024-05-13"
        }:
            tokens_per_message = 3
            tokens_per_name = 1
        elif model == "gpt-3.5-turbo-0301":
            tokens_per_message = 4
            tokens_per_name = -1
        elif "gpt-3.5-turbo" in model:
            self.log += 'Внимание! gpt-3.5-turbo может обновиться. Используйте gpt-3.5-turbo-0613.\n'
            return self.num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
        elif "gpt-4" in model:
            self.log += 'Внимание! gpt-4 может обновиться. Используйте gpt-4-0613.\n'
            return self.num_tokens_from_messages(messages, model="gpt-4-0613")
        else:
            raise NotImplementedError(f"num_tokens_from_messages() не реализован для {model}.")

        # Подсчет токенов
        num_tokens = 0
        for message in messages:
            num_tokens += tokens_per_message
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":
                    num_tokens += tokens_per_name
        return num_tokens + 3  # Добавление токенов для ответа

    def answer_index(self, system, topic, temp=1):
        if not self.search_index:
            self.log += 'Модель необходимо обучить!\n'
            return ''

        docs = self.search_index.similarity_search(topic, k=8)  # Выборка документов
        top_k = 5
        cross_encoder_model = 'cross-encoder/ms-marco-MiniLM-L-6-v2'  # Cross Encoder
        cross_encoder = CrossEncoder(cross_encoder_model)

        pairs = [(topic, doc.page_content) for doc in docs]  # Составление пар
        scores = np.array(cross_encoder.predict(pairs))  # Оценка пар

        top_indices = np.argsort(scores)[-top_k:][::-1]  # Отбор топ 5
        top_scores = scores[top_indices]
        top_docs = [docs[i] for i in top_indices]

        self.log += 'Выбираем документы по схожести:\n'
        message_content = re.sub(r'\n{2}', ' ', '\n '.join([
            f'Отрывок документа №{i + 1}:\n Оценка: {top_scores[i]}\n' + doc.page_content + '\\n'
            for i, doc in enumerate(top_docs)
        ]))
        self.log += f'{message_content}\n'

        messages = [
            {"role": "system", "content": system + f"{message_content}"},
            {"role": "user", "content": topic}
        ]

        self.log += f"\n\nТокенов использовано на вопрос: {self.num_tokens_from_messages(messages, self.model)}\n"

        completion = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=temp
        )

        self.log += '\nСтатистика по токенам:\n'
        self.log += f'Токенов использовано (вопрос): {completion.usage.prompt_tokens}\n'
        self.log += f'Токенов использовано (вопрос-ответ): {completion.usage.total_tokens}\n'

        return completion.choices[0].message.content  # Результат


##Интерфейс gradio

In [ ]:
# Создаем экземпляр класса GPT с моделью LLM
gpt = GPT("gpt-3.5-turbo")

# Gradio объединяет элементы в блоки
blocks = gr.Blocks()

# Работаем с блоком
with blocks as demo:
    subject = gr.Dropdown([(elem["name"], index) for index, elem in enumerate(models)], label="Данные")  # Выбор данных
    name = gr.Label(show_label=False)  # Отображение имени
    prompt = gr.Textbox(label="Промт", interactive=True)  # Промпт для запроса
    link = gr.HTML()  # Ссылка на файл обучения
    query = gr.Textbox(label="Запрос к LLM", interactive=True)  # Поле пользовательского запроса

    # Функция обработки изменения выбора нейро-сотрудника
    def onchange(dropdown):
        return [
            models[dropdown]['name'],  # Имя без изменений
            re.sub('\t+|\s\s+', ' ', models[dropdown]['prompt']),  # Удаление двойных пробелов и табуляции
            models[dropdown]['query'],  # Запрос без изменений
            f"<a target='_blank' href='{models[dropdown]['doc']}'>Документ для обучения</a>"  # Ссылка на документ
        ]

    subject.change(onchange, inputs=[subject], outputs=[name, prompt, query, link])  # Обработчик изменений

    # Разметка кнопок
    with gr.Row():
        train_btn = gr.Button("Обучить модель")  # Кнопка для обучения
        request_btn = gr.Button("Запрос к модели")  # Кнопка для запроса

    # Функция для обучения
    def train(dropdown):
        gpt.load_search_indexes(models[dropdown]['doc'])  # Парсинг документа
        return gpt.log

    # Функция запроса к языковой модели
    def predict(p, q):
        result = gpt.answer_index(p, q)  # Запрос к LLM
        return [result, gpt.log]  # Возврат результата и логов

    # Вывод полей ответа и логирования
    with gr.Row():
        response = gr.Textbox(label="Ответ LLM")  # Ответ от LLM
        log = gr.Textbox(label="Логирование")  # Логи классов GPT

    # Обработчики нажатий кнопок
    train_btn.click(train, [subject], log)  # Кнопка обучения
    request_btn.click(predict, [prompt, query], [response, log])  # Кнопка запроса

# Запуск приложения
demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://90f5626b1268551afe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
